# Case 3. Using word embeddings
Neural networks for Health Technology Applications<br>
26.2.2020, Sakari Lukkarinen<br>
[Helsinki Metropolia University of Applied Sciences](www.metropolia.fi/en)

## Introduction

The aim of this Notebook is to work as introduction to text preprocessing functions for neural networks.

## Acknowledgments

The dataset is from: [UCI ML Drug Review dataset](https://www.kaggle.com/jessicali9530/kuc-hackathon-winter-2018).

![](http://)## Import libraries and read the datasets

In [ ]:
# Read the basic libraries (similar start as in Kaggle kernels)
%pylab inline
import time # for timing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split # preprocessing datasets
from tensorflow.keras.preprocessing.text import Tokenizer # text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences # text preprocessing
from tensorflow.keras.models import Sequential # modeling neural networks
from tensorflow.keras.layers import Dense, Activation # layers for neural networks
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score # final metrics

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("../input"))
tf.__version__

In [ ]:
# Change the default figure size
plt.rcParams['figure.figsize'] = [12, 5]

In [ ]:
# Create dataframes train and test
train = pd.read_csv('../input/drugsComTrain_raw.csv')
test = pd.read_csv('../input/drugsComTest_raw.csv')

# Show the first 5 rows of the train set
train.head()

## Text processing

More info: 
- [scikit-learn CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [scikit-learn text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
- [keras Tokenizer]()


In [ ]:
%%time
# Tokenize the text
samples = train['review']
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(samples)
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


In [ ]:
data = pad_sequences(sequences, maxlen=200)

In [ ]:
%%time
# Create three categories
# label = 4, when rating == 10
# label = 3, when rating == 8...9
# label = 2, when rating = 5..7
# label = 1, when rating = 2..4
# label = 0, when rating = 1
labels = train['rating'].values
for i in range(len(labels)):
    x = labels[i]
    if x == 10:
        labels[i] = 4
    elif x >= 8:
        labels[i] = 3
    elif x >= 5:
        labels[i] = 2
    elif x >= 2:
        labels[i] = 1
    else:
        labels[i] = 0

In [ ]:
from tensorflow.keras.utils import to_categorical

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

In [ ]:
VALIDATION_SPLIT = 0.25

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

## Model

In [ ]:
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant


In [ ]:
embedding_layer = Embedding(5000,
                            100,
                            input_length=200,
                            trainable=True)

sequence_input = Input(shape=(200,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(5, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.summary()

## Training

In [ ]:
%%time
history = model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

In [ ]:
# Plot the accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
e = arange(len(acc)) + 1

plot(e, acc, label = 'train')
plot(e, val_acc, label = 'validation')
title('Training and validation accuracy')
xlabel('Epoch')
grid()
legend()

figure()

plot(e, loss, label = 'train')
plot(e, val_loss, label = 'validation')
title('Training and validation loss')
xlabel('Epoch')
grid()
legend()

show()

## Calculate metrics

In [ ]:
# Find the predicted values for the validation set
y_pred = argmax(model.predict(x_val), axis = 1)
y_true = argmax(y_val, axis = 1)

In [ ]:
# Calculate the classification report
cr = classification_report(y_true, y_pred)
print(cr)

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred).T
print(cm)

In [ ]:
# Calculate the cohen's kappa, both with linear and quadratic weights
k = cohen_kappa_score(y_true, y_pred)
print(f"Cohen's kappa (linear)    = {k:.3f}")
k2 = cohen_kappa_score(y_true, y_pred, weights = 'quadratic')
print(f"Cohen's kappa (quadratic) = {k2:.3f}")


More info: 
- [sklearn.metrics.cohen_kappa_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html)
- [Cohen's kappa (Wikipedia)](https://en.wikipedia.org/wiki/Cohen%27s_kappa)

## Next step
To use Glove word embeddings to fix the embedded layer. For more details see:
    https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [ ]:
_ = """
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
"""